# Import Packages

In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import time
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import re
import seaborn as sns
import matplotlib.animation as animation
import plotly.express as px
import scipy
import math

# Set Notebook Options

In [2]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 100)

# Define Functions

### Play Plot

In [3]:
def play_plot(playID,gameID,df):
    ex_play = df[df['gameId'] == str(gameID)]
    ex_play = ex_play[ex_play['playId'] == str(playID)]   
    fig = px.scatter(ex_play, x="x", y="y",color='team',color_discrete_map={'home': 'rgb(255,60,0)', 'away': 'rgb(238,173,30)', 'football': 'rgb(139,69,19)'},size='size',size_max=13, animation_frame="frameId",range_x=[0,120],range_y=[0,160/3],title=str(ex_play['playDescription'].iloc[0]),text='jerseyNumber',opacity=1)
    fig.add_shape(type="line",x0=10, y0=0, x1=10, y1=160/3,line=dict(color="White",width=5),layer="below")
    fig.add_shape(type="line",x0=110, y0=0, x1=110, y1=160/3,line=dict(color="White",width=5),layer="below")
    for i in range(20,110,10):
        fig.add_shape(type="line",x0=i, y0=0, x1=i, y1=160/3,line=dict(color="White",width=1),layer="below")
    for i in range(15,110,10):
        fig.add_shape(type="line",x0=i, y0=0, x1=i, y1=160/3,line=dict(color="White",width=.5),layer="below")
    for i in range(11,110,1):
        fig.add_shape(type="line",x0=i, y0=29.73, x1=i, y1=30.39,line=dict(color="White",width=.5),layer="below")
        fig.add_shape(type="line",x0=i, y0=22.91, x1=i, y1=23.57,line=dict(color="White",width=.5),layer="below")
        fig.add_shape(type="line",x0=i, y0=0.0, x1=i, y1=0.5,line=dict(color="White",width=.5),layer="below")
        fig.add_shape(type="line",x0=i, y0=52.5, x1=i, y1=53,line=dict(color="White",width=.5),layer="below")
    fig.add_shape(type="rect",x0=110, y0=0, x1=120, y1=160/3,line=dict(color="White",width=1,),fillcolor='rgb(97, 61, 25)',opacity=1,layer="below")
    fig.add_shape(type="rect",x0=0, y0=0, x1=10, y1=160/3,line=dict(color="White",width=1,),fillcolor='rgb(97, 61, 25)',opacity=1,layer="below")
    fig.update_layout({"plot_bgcolor": "rgba(0, 169, 0, 1)","paper_bgcolor": "rgba(0, 0, 0, 0)",})
    fig.update_xaxes(showgrid=True,visible=True, showticklabels=False)
    fig.update_yaxes(showgrid=False,visible=False, showticklabels=False)
    fig.add_annotation(x=116.5, y=20,text=ex_play['homeTeamAbbr'].iloc[0],textangle=90,align="center",valign="middle",font = dict(color = '#FFFFFF',
                                  family = 'sans serif',
                                  size = 40))
    fig.add_annotation(x=6.5, y=20,text=ex_play['homeTeamAbbr'].iloc[0],textangle=-90,align="center",valign="middle",font = dict(color = '#FFFFFF',
                                  family = 'sans serif',
                                  size = 40))
    for r,q in zip(range(10,100,10),[1,2,3,4,5,4,3,2,1]):
        fig.add_annotation(x=r+9, y=13,textangle=0,text=q,align="center",valign="middle",font = dict(color = '#FFFFFF',size = 17),showarrow=False)
        fig.add_annotation(x=r+11, y=13,textangle=0,text="0",align="center",valign="middle",font = dict(color = '#FFFFFF',size = 17),showarrow=False)
        fig.add_annotation(x=r+9, y=160/3-13,textangle=0,text=q,align="center",valign="middle",font = dict(color = '#FFFFFF',size = 17),showarrow=False)
        fig.add_annotation(x=r+11, y=160/3-13,textangle=0,text="0",align="center",valign="middle",font = dict(color = '#FFFFFF',size = 17),showarrow=False)
    fig.update_layout(transition_duration=3000,showlegend=False,xaxis_title=f"{ex_play['visitorTeamAbbr'].iloc[0]} @ {ex_play['homeTeamAbbr'].iloc[0]}  Week {ex_play['week'].iloc[0]}, {ex_play['gameDate'].iloc[0].date()}. PlayID: {ex_play['playId'].iloc[0]} GameID: {ex_play['gameId'].iloc[0]}")
    fig.show()

### Change Orientation so all plays are going the same direction

In [6]:
def new_orientation(angle, play_direction):
    if play_direction == 0:
        new_angle = 360.0 - angle
        if new_angle == 360.0:
            new_angle = 0.0
        return new_angle
    else:
        return angle

# Mapping Data

In [5]:
Opositions = ['OL','QB', 'RB', 'TE', 'WR','FB','HB']
Dpositions = ['DL', 'LB', 'DB','SS','FS','MLB','CB','OLB', 'ILB','NT', 'S', 'DE']

# Import Files

In [4]:
games = pd.read_csv("nfl-big-data-bowl-2021/games.csv",dtype={'gameId': 'object'})
players = pd.read_csv("nfl-big-data-bowl-2021/players.csv")
plays = pd.read_csv("nfl-big-data-bowl-2021/plays.csv",dtype={'playId': 'object','gameId': 'object'})
tracking = pd.DataFrame()
for i in range(1,18):
    print(i)
    df = pd.read_csv("nfl-big-data-bowl-2021/week" + str(i) + ".csv",dtype={'playId': 'object','gameId': 'object'})
    tracking = tracking.append(df)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17


# Clean Games

- Game dates are imported in object format. Here we convert to datetime

In [8]:
games['gameDate'] = pd.to_datetime(games['gameDate']).dt.normalize()

# Clean Players

- nflId is imported as an integer. This is problematic as it can eliminate leading or trailing zeros in the ID. We convert it to a string
- Height information is clean by extracting height in feet, inches, or otherwise. We create a uniform measurement where all players are measured in inches. Feet and Inches columns are then dropped as they are only helpers to create new height.
- birthdate is imported as an object. We convert to datetime.
- Side indicates whether the player primarily plays offense or defense.

In [10]:
players['nflId'] = players['nflId'].astype(str)
players['feet'] = pd.to_numeric(np.where(players['height'].str.contains('-'),players['height'].str.split('-').str[0],np.nan))
players['inches'] = pd.to_numeric(np.where(players['height'].str.contains('-'),players['height'].str.split('-').str[1],np.nan))
players['height'] = np.where(players['height'].str.contains('-'),players['feet']*12+players['inches'],players['height'])
players['height'] = players['height'].astype(int)
players = players.drop(['feet','inches'],axis='columns')
players['birthDate'] = pd.to_datetime(players['birthDate'])
players['side'] = np.where(players['position'].isin(Opositions),'Offense',np.nan)
players['side'] = np.where(players['position'].isin(Dpositions),'Defense',players['side'])

# Clean Plays

- playType has unnecessary text in front of the type of play. This is removed. 
- Apprently we are only analyzing plays in which there are not penalties
- A popular way to describe offensive football is to describe the formation. Formation is not given, but we can impute it based on the assigned positions of players on the field. it is possible that a WR could lineup in the backfield, or vice versa, but this is a general sense of formations used. 
- Only plays which have 11 confirmed players are retained in the dataset. Outliers may be explored at a later date. 
- Defensive formation is imputed in similar fashion to offense. 
- Again, only plays are retained in which there are 11 confirmed players.
- Defenders in the box, home and away score are converted to an integer

In [12]:
plays['playType'] = plays['playType'].str.replace("play_type_","")
plays = plays[plays['penaltyCodes'].isna()]
for p in Opositions:
    pattern = '(..' + str(p) + ')'
    plays[p] = 0
    plays[p] = plays['personnelO'].str.extract(pat=pattern,expand=True)
    plays[p] = plays[p].str.replace(r'\D+', '').fillna(0).astype(int)
plays['OL'] = np.where(plays['OL'] == 0, 5, plays['OL'])
plays['QB'] = np.where(plays['QB'] == 0, 1, plays['QB'])
plays['Off'] = plays['QB'] + plays['RB'] + plays['WR'] + plays['TE'] + plays['OL']
plays = plays[plays['Off'] == 11]

for p in Dpositions:
    pattern = '(..' + str(p) + ')'
    plays[p] = 0
    plays[p] = plays['personnelD'].str.extract(pat=pattern,expand=True)
    plays[p] = plays[p].str.replace(r'\D+', '').fillna(0).astype(int)

plays['Def'] = plays['DL'] + plays['LB'] + plays['DB']
plays = plays[plays['Def'] == 11]
plays = plays[plays['defendersInTheBox'].notna()]
plays['defendersInTheBox'] = plays['defendersInTheBox'].astype(int)
plays = plays[plays['numberOfPassRushers'].notna()]
plays['numberOfPassRushers'] = plays['numberOfPassRushers'].astype(int)
plays = plays[plays['preSnapVisitorScore'].notna()]
plays['preSnapVisitorScore'] = plays['preSnapVisitorScore'].astype(int)
plays['preSnapHomeScore'] = plays['preSnapHomeScore'].astype(int)
plays['personnel'] = plays['RB'].astype(str) + plays['TE'].astype(str)

# Clean Tracking

- Players are assigned unigue IDs. The football itself is included in tracking data, however it is not assigned an ID. Because this can cause issues in future data cleaning, the football is simply given and identification number of 0.
- nflId is converted to an integer and then a string......
- play direction is unified so that all plays are moving in the same direciton. 

In [13]:
tracking['nflId'] = np.where(tracking['nflId'].notnull(),tracking['nflId'],0)
tracking['nflId'] = tracking['nflId'].astype(int).astype(str)
tracking.dropna(subset=["playDirection"],inplace=True)
tracking['playDirection'] = tracking['playDirection'].apply(lambda x: x.strip() == 'right')
tracking['x'] = tracking.apply(lambda row: row['x'] if row['playDirection'] else 120-row['x'], axis=1)
tracking['y'] = tracking.apply(lambda row: row['y'] if row['playDirection'] else 160/3-row['y'], axis=1)
tracking['o'] = tracking.apply(lambda row: new_orientation(row['o'], row['playDirection']), axis=1)
tracking['dir'] = tracking.apply(lambda row: new_orientation(row['dir'], row['playDirection']), axis=1)

### Join Datasets
- Joining plays, games, and tracking data. The purpose of this to get like information into the same dataframe for plotting
- 'size' is a dummy variable for plotting the size of dots on the graph. We want the football to appear smaller than the players

In [14]:
joined = plays[['gameId', 'playId', 'playDescription']]
joined = pd.merge(joined,games,on='gameId',how='inner')
joined = pd.merge(joined,tracking,on=['gameId','playId'],how='inner')
joined['size'] = np.where(joined['team'] == 'football',2,5)

- The creation of this ID allows each frame of each play to be identified uniquely.

In [15]:
joined['ID'] = joined['gameId'].astype(str) + joined['playId'].astype(str) + joined['frameId'].astype(str)
joined['ID'] = pd.to_numeric(joined['ID'])

# Plotting
- This pulls one game at random from the sample.
- Then, from that game, pulls a play at random and plots it

In [17]:
game = joined['gameId'].sample(1).iloc[0]
my_game = joined[joined['gameId'] == game]
my_play = my_game['playId'].sample(1).iloc[0]
print(my_play,game)
play_plot(my_play,game,joined)

3492 2018111105


# Perform analysis on one play

- We know take the randomly selected play and begin to perform analysis on it

In [18]:
one_play = joined[joined['playId'] == my_play]
one_play = one_play[one_play['gameId'] == game]
one_play = one_play['ID'].unique()

- For each play data is captured at 0.1 second intervals and uniquely labeled with frameIds
- As we loop through each Frame, the following actions take place:
    - Record how far apart all players are from each other and the football using euclidean distance from the scipy package.
    - pairing the closest offensive player to each player
    - pairing the closest defensive player with each player
 

In [20]:
df_next = pd.DataFrame()
c = 1
#for i in joined['ID']:
for i in one_play:
    ex_play_demo = joined[joined['ID'] == i]
    ex_play_demo = ex_play_demo.copy()
    mat = scipy.spatial.distance.cdist(ex_play_demo[['x','y']], ex_play_demo[['x','y']], metric='euclidean')
    new_df = pd.DataFrame(mat, index=ex_play_demo[['nflId','team']], columns=ex_play_demo[['nflId','team']]) 
    test = new_df.stack().reset_index()
    test = test.rename(columns={'level_0':'player_1','level_1':'player_2',0:'distance'})
    test['player_1'] = test['player_1'].agg(','.join)
    test['player_2'] = test['player_2'].agg(','.join)
    test[['player_1','team_1']] = test['player_1'].str.split(",",expand=True)
    test[['player_2','team_2']] = test['player_2'].str.split(",",expand=True)
    test = test.drop_duplicates(keep='first')
    test['away_distance'] = np.where((test['team_2'] == 'away'),test['distance'],np.nan)
    test['home_distance'] = np.where((test['team_2'] == 'home'),test['distance'],np.nan)
    test['away_distance'] = np.where((test['away_distance'] == 0),np.nan,test['away_distance'])
    test['home_distance'] = np.where((test['home_distance'] == 0),np.nan,test['home_distance'])
    test = test.dropna(subset=['away_distance','home_distance'],how='all')
    test2 = test.groupby('player_1').agg({'away_distance':'min','home_distance':'min'}).reset_index()
    test2 = pd.merge(test2,test[['player_1','team_1','player_2','team_2','home_distance']],how='left',on=['player_1','home_distance'],suffixes=['','_home']).drop_duplicates(keep='first')
    test2 = pd.merge(test2,test[['player_1','team_1','player_2','team_2','away_distance']],how='left',on=['player_1','away_distance'],suffixes=['','_away']).drop_duplicates(keep='first')
    del test2['team_1_away']
    test2 = pd.merge(test2,players[['nflId','side']],left_on='player_1',right_on='nflId',suffixes=['','_player_1'],how='left').drop_duplicates(keep='first')
    test2 = pd.merge(test2,players[['nflId','side']],left_on='player_2',right_on='nflId',suffixes=['','_player_2'],how='left').drop_duplicates(keep='first')
    test2 = pd.merge(test2,players[['nflId','side']],left_on='player_2_away',right_on='nflId',suffixes=['','_player_2_away'],how='left').drop_duplicates(keep='first')
    test2['closest_offense'] = np.where((test2['side_player_2_away'] == 'Offense'),test2['away_distance'],test2['home_distance'])
    test2['closest_defense'] = np.where((test2['side_player_2_away'] == 'Offense'),test2['home_distance'],test2['away_distance'])
    test2['closest_offense_ID'] = np.where((test2['side_player_2_away'] == 'Offense'),test2['player_2_away'],test2['player_2'])
    test2['closest_defense_ID'] = np.where((test2['side_player_2_away'] == 'Offense'),test2['player_2'],test2['player_2_away'])
    test2 = test2[['player_1','closest_offense_ID','closest_offense','closest_defense_ID','closest_defense']]
    test2 = pd.merge(test2,players[['nflId','displayName']],left_on='player_1',right_on='nflId',how='left')
    del test2['nflId']
    test2 = pd.merge(test2,players[['nflId','displayName']],left_on='closest_offense_ID',right_on='nflId',how='left')
    del test2['nflId']
    test2 = pd.merge(test2,players[['nflId','displayName']],left_on='closest_defense_ID',right_on='nflId',how='left')
    del test2['nflId']
    test2 = test2.rename(columns={'displayName_x':'player_1_name','displayName_y':'closest_offense_name','displayName':'closest_defense_name'})
    merge = pd.merge(ex_play_demo,test2,left_on='nflId',right_on='player_1',how='left')
    df_next = df_next.append(merge)
    print(c)
    c = c+1

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56


- ball_snap: the frame in which the ball is snapped 
- pass_forward: the frame in which the passer throws the ball forward
- pass_arrived: the frame in which the ball arrives to the receiver
- gameID: the gameID of the game we are reviewing
- targeted_wr: the closest offensive player to the football at the time the pass arrives
- targeted_db: the closest defensive player to the football at the time the pass arrives
- ball_x: the x coordinate which the ball arrives at
- ball_y:the y coordinate which the ball arrives at

In [21]:
ball_snap = df_next.loc[df_next['event'] == 'ball_snap', 'ID'][0]
pass_forward = df_next.loc[df_next['event'] == 'pass_forward', 'ID'][0]
pass_arrived = df_next.loc[df_next['event'] == 'pass_arrived', 'ID'][0]
gameID = df_next.loc[df_next['event'] == 'ball_snap', 'gameId'][0]
targeted_wr = df_next.loc[(df_next['event'] == 'pass_arrived')&(df_next['displayName'] == 'Football'), 'closest_offense_ID'].values[0]
targeted_db = df_next.loc[(df_next['event'] == 'pass_arrived')&(df_next['displayName'] == 'Football'), 'closest_defense_ID'].values[0]
ball_x = df_next.loc[(df_next['event'] == 'pass_arrived')&(df_next['displayName'] == 'Football'), 'x'].values[0]
ball_y = df_next.loc[(df_next['event'] == 'pass_arrived')&(df_next['displayName'] == 'Football'), 'y'].values[0]

- event2: creates deeper detail to the event field, recording when the QB is dropping back and when the ball is in flight.

In [22]:
df_next['event2'] = np.where(df_next['ID'] < ball_snap,'pre_snap',df_next['event'])
df_next['event2'] = np.where((df_next['ID'] > ball_snap)&(df_next['ID'] < pass_forward),'dropback',df_next['event2'])
df_next['event2'] = np.where((df_next['ID'] > pass_forward)&(df_next['ID'] < pass_arrived),'ball_in_flight',df_next['event2'])
df_next['targeted_wr'] = targeted_wr
df_next['targeted_db'] = targeted_db

- off_tar: filters to frames only for the targeted offensive player
- def_tar: filters to frames only for the targeted defensive player
- now: a dataframe which highlights the coordinates of the targeted players compared specifically to each other at each frame in the play.
- OD: creates a column which shows how far apart the targeted players are at each frame
- OF: compares distance of offensive target to ball at each frame of the play
- DF: compares distance of defensive target to ball at each frame of the play
- Finally, the data is merged back into the original dataframe.

In [23]:
off_tar = df_next[df_next['player_1'] == df_next['targeted_wr']]
def_tar = df_next[df_next['player_1'] == df_next['targeted_db']]
now = pd.merge(off_tar[['frameId','player_1','x','y']],def_tar[['frameId','player_1','x','y']],on='frameId',how='inner',suffixes=['_off','_def'])
now['ball_arrive_x'] = ball_x
now['ball_arrive_y'] = ball_y
now['diff_OD'] = np.sqrt(((now['x_def'] - now['x_off'])**2)+((now['y_def'] - now['y_off'])**2))
now['diff_OF'] = np.sqrt(((now['ball_arrive_x'] - now['x_off'])**2)+((now['ball_arrive_y'] - now['y_off'])**2))
now['diff_DF'] = np.sqrt(((now['ball_arrive_x'] - now['x_def'])**2)+((now['ball_arrive_y'] - now['y_def'])**2))
df_next = pd.merge(df_next,now[['frameId','diff_OD','diff_OF','diff_DF']],how='left',on='frameId')

- play_developing: a binary flag showing whether they play is in development
- ovo_coverage: a binary flag stating whether the targeted players are closest to each other at each frame
- true_1v1: identifies if the targeted offender and defender are closest to each other at each frame of the developing play

In [24]:
df_next['play_developing'] = np.where((df_next['ID'] >= ball_snap)&(df_next['ID'] <= pass_arrived)&(df_next['player_1'] == targeted_wr),1,0)
df_next['ovo_coverage'] = np.where((df_next['play_developing'] == 1)&(df_next['player_1'] == targeted_wr)&(df_next['closest_defense_ID'] == targeted_db),1,0)
df_next['true_1v1_play'] = np.where((df_next['play_developing'] == 1)&(df_next['ovo_coverage'] == 1),1,0)
if df_next['play_developing'].sum() == df_next['ovo_coverage'].sum():
    df_next['true_1v1_play'] == 1
else:
    df_next['true_1v1_play'] == 1

- The next line creates a simple score which highlights how far the defender is from the target location at the ball snap, pass release, and pass arrival. This will eventually be used to highlight player anticipation skills

In [25]:
df_next['targeted_db_score'] = (100 - df_next['diff_DF']) / 100

In [27]:
df_next[df_next['player_1'] == targeted_wr]

,gameId,playId,playDescription,gameDate,gameTimeEastern,homeTeamAbbr,visitorTeamAbbr,week,time,x,y,s,a,dis,o,dir,event,nflId,displayName,jerseyNumber,position,frameId,team,playDirection,route,size,ID,player_1,closest_offense_ID,closest_offense,closest_defense_ID,closest_defense,player_1_name,closest_offense_name,closest_defense_name,event2,targeted_wr,targeted_db,diff_OD,diff_OF,diff_DF,play_developing,ovo_coverage,true_1v1_play,targeted_db_score
9,2018111105,3492,"(4:31) (No Huddle, Shotgun) J.Rosen pass short...",2018-11-11,13:00:00,KC,ARI,10,2018-11-11T20:40:29.900Z,49.00,17.69,0.00,0.00,0.00,73.85,155.04,None,2559248,Ricky Seals-Jones,86.0,TE,1,away,True,OUT,5,201811110534921,2559248,2561272,6.761139,2550257,3.796169,Ricky Seals-Jones,Trent Sherfield,Daniel Sorensen,pre_snap,2559248,2550257,3.796169,10.940603,8.776018,0,0,0,0.912240
24,2018111105,3492,"(4:31) (No Huddle, Shotgun) J.Rosen pass short...",2018-11-11,13:00:00,KC,ARI,10,2018-11-11T20:40:30.000Z,48.99,17.69,0.00,0.00,0.00,73.85,155.93,None,2559248,Ricky Seals-Jones,86.0,TE,2,away,True,OUT,5,201811110534922,2559248,2561272,6.758713,2550257,3.795260,Ricky Seals-Jones,Trent Sherfield,Daniel Sorensen,pre_snap,2559248,2550257,3.795260,10.948119,8.772463,0,0,0,0.912275
39,2018111105,3492,"(4:31) (No Huddle, Shotgun) J.Rosen pass short...",2018-11-11,13:00:00,KC,ARI,10,2018-11-11T20:40:30.099Z,48.99,17.69,0.00,0.00,0.00,72.21,157.08,None,2559248,Ricky Seals-Jones,86.0,TE,3,away,True,OUT,5,201811110534923,2559248,2561272,6.758713,2550257,3.796169,Ricky Seals-Jones,Trent Sherfield,Daniel Sorensen,pre_snap,2559248,2550257,3.796169,10.948119,8.781082,0,0,0,0.912189
54,2018111105,3492,"(4:31) (No Huddle, Shotgun) J.Rosen pass short...",2018-11-11,13:00:00,KC,ARI,10,2018-11-11T20:40:30.200Z,48.99,17.69,0.00,0.00,0.00,72.21,157.56,line_set,2559248,Ricky Seals-Jones,86.0,TE,4,away,True,OUT,5,201811110534924,2559248,2561272,6.757522,2550257,3.787149,Ricky Seals-Jones,Trent Sherfield,Daniel Sorensen,pre_snap,2559248,2550257,3.787149,10.948119,8.794771,0,0,0,0.912052
69,2018111105,3492,"(4:31) (No Huddle, Shotgun) J.Rosen pass short...",2018-11-11,13:00:00,KC,ARI,10,2018-11-11T20:40:30.299Z,48.99,17.70,0.00,0.00,0.00,72.21,157.40,None,2559248,Ricky Seals-Jones,86.0,TE,5,away,True,OUT,5,201811110534925,2559248,2561272,6.766277,2550257,3.778161,Ricky Seals-Jones,Trent Sherfield,Daniel Sorensen,pre_snap,2559248,2550257,3.778161,10.954716,8.817080,0,0,0,0.911829
84,2018111105,3492,"(4:31) (No Huddle, Shotgun) J.Rosen pass short...",2018-11-11,13:00:00,KC,ARI,10,2018-11-11T20:40:30.400Z,48.99,17.69,0.00,0.00,0.00,72.21,158.91,None,2559248,Ricky Seals-Jones,86.0,TE,6,away,True,OUT,5,201811110534926,2559248,2561272,6.766277,2550257,3.779153,Ricky Seals-Jones,Trent Sherfield,Daniel Sorensen,pre_snap,2559248,2550257,3.779153,10.948119,8.817080,0,0,0,0.911829
99,2018111105,3492,"(4:31) (No Huddle, Shotgun) J.Rosen pass short...",2018-11-11,13:00:00,KC,ARI,10,2018-11-11T20:40:30.500Z,48.99,17.69,0.00,0.00,0.00,72.21,158.45,None,2559248,Ricky Seals-Jones,86.0,TE,7,away,True,OUT,5,201811110534927,2559248,2561272,6.765116,2550257,3.779153,Ricky Seals-Jones,Trent Sherfield,Daniel Sorensen,pre_snap,2559248,2550257,3.779153,10.948119,8.817080,0,0,0,0.911829
114,2018111105,3492,"(4:31) (No Huddle, Shotgun) J.Rosen pass short...",2018-11-11,13:00:00,KC,ARI,10,2018-11-11T20:40:30.599Z,49.00,17.69,0.00,0.00,0.00,72.21,155.84,None,2559248,Ricky Seals-Jones,86.0,TE,8,away,True,OUT,5,201811110534928,2559248,2561272,6.766277,2550257,3.767240,Ricky Seals-Jones,Trent Sherfield,Daniel Sorensen,pre_snap,2559248,2550257,3.767240,10.940603,8.799847,0,0,0,0.912002
129,2018111105,3492,"(4:31) (No Huddle, Shotgun) J.Rosen pass short...",2018-11-11,13:00:00,KC,ARI,10,2018-11-11T20:40:30.700Z,49.00,17.69,0.00,0.11,0.00,72.21,59.50,None,2559248,Ricky Seals-Jones,86.0,TE,9,away,True,OUT,5,201811110534929,2559248,2561272,6.776208,2550257,3.767240,Ricky Seals-Jones,Trent Sherfield,Daniel Sorensen,pre_snap,2559248,25502